In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
#read train and test data
train_data = pd.read_excel('训练.xlsx')
test_data_1 = pd.read_excel('测试A.xlsx')
test_data_2 = pd.read_excel('测试B.xlsx')

In [3]:
train_nonnull = train_data.isnull().values
test_nonnull_1 = test_data_1.isnull().values
test_nonnull_2 = test_data_2.isnull().values
train_X = train_data.values
test_X_1 = test_data_1.values
test_X_2 = test_data_2.values
test_id_1 = test_X_1[:, 0]
test_id_2 = test_X_2[:, 0]
train_y = train_X[:, -1]
train_X = train_X[:, 1:-1]
test_X_1 = test_X_1[:, 1:]
test_X_2 = test_X_2[:, 1:]

In [4]:
train_X.shape, test_X_1.shape, test_X_2.shape

((500, 8027), (100, 8027), (121, 8027))

In [5]:
#Merge two data
merged_X = np.concatenate((train_X, test_X_1, test_X_2), axis=0)
merged_nn = np.concatenate((train_nonnull[:, 1:-1], test_nonnull_1[:, 1:], test_nonnull_2[:, 1:]), axis=0)
print(merged_X.shape, merged_nn.shape)

(721, 8027) (721, 8027)


In [6]:
#reverse nn
def BoolReverse(columns):
    tmpm = np.zeros(columns.shape)
    tmpm[columns] = 1
    return tmpm == 0

merged_nn = BoolReverse(merged_nn)

In [7]:
# now find the values that has 0 value and delete such column
first_delete_idxs = []
for i in range(merged_X.shape[1]):
    data = merged_X[:, i]
    real = merged_nn[:, i]
    rd = data[real]
    if len(rd) == 0:
        first_delete_idxs.append(i)
    elif len(set(rd)) == 0:
        first_delete_idxs.append(i)
print(len(first_delete_idxs))

62


In [8]:
print(merged_X.shape)
merged_X = np.delete(merged_X, first_delete_idxs, axis=1)
print(merged_X.shape)

(721, 8027)
(721, 7965)


In [9]:
print(merged_nn.shape)
merged_nn = np.delete(merged_nn, first_delete_idxs, axis=1)
print(merged_nn.shape)

(721, 8027)
(721, 7965)


In [10]:
#now find the columns that has only one value
only_one_values = 0
for i in range(merged_X.shape[1]):
    data = merged_X[:, i]
    real = merged_nn[:, i]
    rd = data[real]
    if len(set(rd)) == 1:
        merged_X[:, i] = 1.0
        merged_nn[:, i] = True
        only_one_values += 1
print(only_one_values)

1007


In [11]:
def dict2list(dic):
    keys = dic.keys()
    vals = dic.values()
    lst = [(key, val) for key, val in zip(keys, vals)]
    return lst

# input one array and return the most frequent one
def MostFrequentOne(column):
    itemfreq = {}
    for i in column:
        if i not in itemfreq:
            itemfreq[i] = 1
        else:
            itemfreq[i] += 1
    tmp_dict = sorted(dict2list(itemfreq), key=lambda d:d[1], reverse=True)
    return tmp_dict[0][0]

In [12]:
# one_hot to map str to int
def OneHot_Str(column, retdict = 0):
    syms = set(column)
    sim = {}
    count = 0
    for s in syms:
        sim[s] = count
        count += 1
    cl = len(column)
    retoh = np.zeros((cl, count))
    for i in range(cl):
        s = column[i]
        mi = sim[s]
        retoh[i, mi] = 1.0
    if retdict == 0:
        return retoh
    else:
        return retoh, sim

In [13]:
#now fix the remain data
#if the values in data has one value that is string, then one_hot it, else normalize it
stridxs = []
for i in range(merged_X.shape[1]):
    data = merged_X[:, i]
    real = merged_nn[:, i]
    rd = data[real]
    #if len(data) == len(rd) and np.mean(data) == 1.0:
    #    continue
    isstr = False
    for j in range(len(rd)):
        x = rd[j]
        sx = str(type(x))
        if 'str' in sx:
            isstr = True
            break
    if isstr:
        stridxs.append(i)
        # fix null and one_hot it
        if len(rd) < len(data):
            mfc = MostFrequentOne(rd)
            rn = BoolReverse(real)
            data[rn] = mfc
        onehots = OneHot_Str(data)
        merged_X = np.concatenate((merged_X, onehots), axis=1)
    else:
        if len(rd) < len(data):
            rn = BoolReverse(real)
            data[rn] = np.mean(rd)
            mean = np.mean(data)
            std = np.std(data)
            data = (data-mean)/std
            merged_X[:, i] = data
#delete the string columns
print(len(stridxs))
print(merged_X.shape)
merged_X = np.delete(merged_X, stridxs, axis=1)
print(merged_X.shape)

10
(721, 7999)
(721, 7989)


In [14]:
#now use pca to reduce its demension
from sklearn.decomposition import PCA

In [15]:
cn = 15 
pca = PCA(n_components=cn) 
pca.fit(merged_X) 
psum = 0 
pe = pca.explained_variance_ratio_
for i in range(cn): 
    e = pe[i] 
    psum += e 
    print(i, psum) 
X_new = pca.transform(merged_X) 
print("new shape", X_new.shape)

0 0.999999975997
1 0.999999988161
2 0.999999996208
3 0.999999999192
4 0.999999999921
5 0.999999999973
6 0.999999999996
7 0.999999999998
8 0.999999999999
9 0.999999999999
10 1.0
11 1.0
12 1.0
13 1.0
14 1.0
new shape (721, 15)


In [16]:
# normalize y
y_max = np.max(train_y)
y_min = np.min(train_y)
real_y = (train_y-y_min)/(y_max-y_min)

In [20]:
# normalize new_X
for i in range(cn):
    col = X_new[:, i]
    m = np.mean(col)
    s = np.std(col)
    col = (col-m)/s
    X_new[:, i] = col

In [21]:
# split train, test1, test2 data
trainX = X_new[:500]
test1X = X_new[500:600]
test2X = X_new[600:]
print(trainX.shape, test1X.shape, test2X.shape)

(500, 15) (100, 15) (121, 15)


In [26]:
# Modeling and predicting
import tensorflow as tf

In [87]:
def model(input_data):
    l1 = tf.layers.dense(input_data, 1)
    return l1

In [88]:
input_X = tf.placeholder(shape=[None, cn], dtype=tf.float32)
input_y = tf.placeholder(shape=[None], dtype=tf.float32)

In [89]:
pred_y = model(input_X)

In [90]:
def loss_f(y, y_pred):
    return tf.reduce_mean(tf.square(y-y_pred))

def MSE(y, y_pred):
    #yp = y_pred*(y_max-y_min)+y_min
    return tf.reduce_mean(tf.square(y-y_pred))

In [91]:
loss = loss_f(input_y, pred_y) 
mse = MSE(input_y, pred_y) 
opt = tf.train.AdamOptimizer(0.001).minimize(loss)

In [92]:
sess = tf.Session()
tf.global_variables_initializer().run(session=sess)

In [101]:
for i in range(1001):
    _, l = sess.run([opt, loss], feed_dict={input_X:trainX, input_y:train_y})
    print(i, l)
    if i % 10 == 0:
        m = sess.run(mse, feed_dict={input_X:trainX, input_y:train_y})
        print(m)

0 0.0426059
0.0425968
1 0.0425968
2 0.0425878
3 0.042579
4 0.0425701
5 0.0425611
6 0.0425521
7 0.0425436
8 0.0425351
9 0.0425264
10 0.0425175
0.0425089
11 0.0425089
12 0.0425001
13 0.0424916
14 0.042483
15 0.0424746
16 0.0424661
17 0.0424577
18 0.0424491
19 0.0424404
20 0.0424323
0.042424
21 0.042424
22 0.0424156
23 0.0424074
24 0.0423992
25 0.042391
26 0.0423828
27 0.0423747
28 0.0423666
29 0.0423583
30 0.0423505
0.0423424
31 0.0423424
32 0.0423344
33 0.0423264
34 0.0423184
35 0.0423105
36 0.0423024
37 0.0422945
38 0.0422867
39 0.0422789
40 0.0422709
0.0422632
41 0.0422632
42 0.0422553
43 0.0422475
44 0.04224
45 0.0422321
46 0.0422246
47 0.0422169
48 0.0422094
49 0.0422019
50 0.0421944
0.0421868
51 0.0421868
52 0.0421792
53 0.0421716
54 0.0421642
55 0.0421569
56 0.0421493
57 0.0421422
58 0.0421348
59 0.0421276
60 0.0421202
0.0421127
61 0.0421127
62 0.0421058
63 0.0420984
64 0.0420913
65 0.0420843
66 0.042077
67 0.0420698
68 0.0420627
69 0.0420556
70 0.0420488
0.0420415
71 0.0420415
72

0.0404728
581 0.0404728
582 0.0404719
583 0.0404712
584 0.0404704
585 0.0404697
586 0.040469
587 0.040468
588 0.040467
589 0.0404662
590 0.0404652
0.0404645
591 0.0404645
592 0.0404636
593 0.040463
594 0.0404621
595 0.0404612
596 0.0404603
597 0.0404593
598 0.0404585
599 0.0404579
600 0.0404571
0.0404564
601 0.0404564
602 0.0404557
603 0.0404549
604 0.0404541
605 0.0404534
606 0.0404526
607 0.040452
608 0.0404511
609 0.0404504
610 0.0404496
0.0404487
611 0.0404487
612 0.0404478
613 0.0404472
614 0.0404464
615 0.0404457
616 0.040445
617 0.0404441
618 0.0404434
619 0.0404428
620 0.040442
0.0404412
621 0.0404412
622 0.0404405
623 0.0404399
624 0.0404393
625 0.0404387
626 0.040438
627 0.0404372
628 0.0404365
629 0.0404358
630 0.0404351
0.0404344
631 0.0404344
632 0.0404338
633 0.0404332
634 0.0404325
635 0.0404318
636 0.0404311
637 0.0404304
638 0.0404295
639 0.040429
640 0.0404284
0.0404276
641 0.0404276
642 0.040427
643 0.0404265
644 0.0404257
645 0.0404253
646 0.0404247
647 0.0404242
64

In [102]:
# get prediction and write the answer
test1_pred = sess.run(pred_y, feed_dict={input_X:test1X})

In [103]:
test1_pred

array([[ 2.84237456],
       [ 2.84236979],
       [ 2.84035826],
       [ 2.8403511 ],
       [ 2.84015203],
       [ 2.84074783],
       [ 2.8388567 ],
       [ 2.83939934],
       [ 2.83939934],
       [ 2.83866572],
       [ 2.83866572],
       [ 2.83990717],
       [ 2.83958435],
       [ 2.83885407],
       [ 2.83991003],
       [ 2.83977866],
       [ 2.83977866],
       [ 2.84034777],
       [ 2.8397789 ],
       [ 2.83976889],
       [ 2.83897781],
       [ 2.83976889],
       [ 2.83976889],
       [ 2.83992815],
       [ 2.83976936],
       [ 2.83860087],
       [ 2.8397696 ],
       [ 2.8397696 ],
       [ 2.83992839],
       [ 2.83929944],
       [ 2.83930421],
       [ 2.83930397],
       [ 2.84009862],
       [ 2.84009981],
       [ 2.83917332],
       [ 2.83917499],
       [ 2.83868575],
       [ 2.8388114 ],
       [ 2.83868575],
       [ 2.83868575],
       [ 2.84010696],
       [ 2.83868146],
       [ 2.83881187],
       [ 2.83849788],
       [ 2.83899808],
       [ 2

In [52]:
test1_pred = test1_pred.reshape(100)
print(test_id_1.shape, test1_pred.shape)

(100,) (100,)


In [53]:
save = pd.DataFrame({'ID':test_id_1, 'value':test1_pred})
save.to_csv('answer.csv', index=False, header=False)

In [56]:
print(y_s, y_m)

0.200769269595 2.8461873043805452


In [59]:
print(np.max(train_y), np.min(train_y))

3.454555830692319 2.3268455638817764


In [81]:
sess.close()

In [17]:
#Regression by machinr learn
def MSE_np(y, y_pred):
    return np.mean(np.square(y-y_pred))

In [18]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [22]:
lr.fit(trainX, train_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [ ]:
test1_pred = lr.predict(test1X)

In [ ]:
test1_pred = test1_pred.reshape(100)
save = pd.DataFrame({'ID':test_id_1, 'value':test1_pred})
save.to_csv('answer.csv', index=False, header=False)

In [ ]:
train_pred = lr.predict(trainX)
mse_tr = MSE_np(train_y, train_pred)
print(mse_tr)

In [24]:
from sklearn.cross_validation import train_test_split

/home/magnusterra/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [34]:
trX, vaX, tr_y, va_y = train_test_split(trainX, train_y, test_size=0.2, random_state=20)

In [35]:
lr = LinearRegression()
lr.fit(trX, tr_y)
tr_p = lr.predict(trX)
print(MSE_np(tr_y, tr_p))

0.036568362863228905


In [36]:
va_p = lr.predict(vaX)
print(MSE_np(va_y, va_p))

0.029323336623688974
